In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
import pandas as pd

2024-10-16 16:21:34.620850: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-16 16:21:34.700681: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-16 16:21:34.700699: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-10-16 16:21:35.111844: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [3]:
data = 'tem'
target_X= pd.read_csv(f"../data/{data}_train_input_7.csv").iloc[:,1:].values.astype(np.float32)
target_y =pd.read_csv(f"../data/{data}_train_output_7.csv").iloc[:,1:].values.astype(np.float32)

X_train = target_X[:-round(target_X.shape[0]*0.2),:].astype(np.float32)
y_train = target_y[:-round(target_y.shape[0]*0.2)].astype(np.float32)

X_train_val= target_X[-round(target_X.shape[0]*0.2):,:].astype(np.float32)
y_train_val =target_y[-round(target_y.shape[0]*0.2):].astype(np.float32)


test_X= pd.read_csv(f"../data/{data}_val_input_7.csv").iloc[:,1:].values.astype(np.float32)
test_y =pd.read_csv(f"../data/{data}_val_output_7.csv").iloc[:,1:].values.astype(np.float32)

#y_train.astype(np.float32)
backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

In [4]:
#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]

#################################################################################
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 unit
    # 4 nhead
    # 5 nlayers
    # dropout
    return X_train.shape[1],y_train.shape[1],84,4,4,0.2

#################################################################################



##########################################################################################
# 트랜스퍼 레이어
class PositionalEncoding(layers.Layer):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = layers.Dropout(rate=dropout)

        position = np.arange(max_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = np.zeros((max_len, d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        pe = pe[np.newaxis, ...]

        self.pe = tf.constant(pe, dtype=tf.float32)

    def call(self, x):
        x = x + self.pe[:, :tf.shape(x)[1], :]
        return self.dropout(x)
##########################################################################################
# 트랜스퍼 레이어
def create_model(fn,d_model, nlayers, nhead, dropout, iw, ow,lr):
    
    
    x = layers.Dense(d_model // 2, activation='relu')(pretrained_output_reshaped)
    x = layers.Dense(d_model, activation='relu')(x)
    
    pos_encoding = PositionalEncoding(d_model, dropout)
    x = pos_encoding(x)
    
    for _ in range(nlayers):
        attn_output = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model, dropout=dropout)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
        ffn_output = layers.Dense(d_model, activation='relu')(x)
        ffn_output = layers.Dense(d_model)(ffn_output)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ffn_output)
    
    x = layers.Dense(d_model // 2, activation='relu')(x)
    x = layers.Dense(1)(x)
    x = tf.squeeze(x, axis=-1)
    
    outputs = layers.Dense((iw + ow) // 2, activation='relu')(x)
    outputs = layers.Dense(ow)(outputs)
    
    optimizer = Adam(learning_rate=lr)
    target_model = Model(inputs=inputs, outputs=outputs)
    target_model.compile(optimizer=optimizer, loss=fn)
    
    return target_model

#################################################################################
# 트랜스포머 모델 생성 함수
def bulid_model(iw, ow, d_model, nhead, nlayers, dropout=0.1):
    inputs = tf.keras.Input(shape=(iw, 1))
    x = layers.Dense(d_model // 2, activation='relu')(inputs)
    x = layers.Dense(d_model, activation='relu')(x)

    pos_encoding = PositionalEncoding(d_model, dropout)
    x = pos_encoding(x)

    for _ in range(nlayers):
        attn_output = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model, dropout=dropout)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
        ffn_output = layers.Dense(d_model, activation='relu')(x)
        ffn_output = layers.Dense(d_model)(ffn_output)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ffn_output)

    x = layers.Dense(d_model // 2, activation='relu')(x)
    x = layers.Dense(1)(x)
    x = tf.squeeze(x, axis=-1)

    outputs = layers.Dense((iw + ow) // 2, activation='relu')(x)
    outputs = layers.Dense(ow)(outputs)

    model = Model(inputs=inputs, outputs=outputs)
    return model

#################################################################################
# 부트스트랩 샘플링
# 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr=0.001):
    models = {}
    iw, ow, d_model, nhead, nlayers, dropout = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(iw, ow, d_model, nhead, nlayers, dropout=0.5)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=0, 
                  callbacks=[early_stop],
               validation_data = [X_train_val,y_train_val])
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys
#################################################################################
# 예측

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions



In [5]:
model_num = 10


mase_models = train_bagging_models(model_num, MASE(y_train,24),2000,10,8,lr=0.001)
mape_models = train_bagging_models(model_num,'mape',2000,10,8,lr=0.001)
smape_models = train_bagging_models(model_num, SMAPE(),2000,10,8,lr=0.001)
mae_models = train_bagging_models(model_num, 'mae',2000,10,8,lr=0.001)
mse_models = train_bagging_models(model_num, 'mse',2000,10,8,lr=0.001)

2024-10-16 16:21:52.679195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-10-16 16:21:52.679229: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-10-16 16:21:52.679715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'########################################################Model5
'########################################################Model6
'########################################################Model7
'########################################################Model8
'########################################################Model9
'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'#######################################

In [6]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions = bagging_predict2(pred1, X_train_val)
mase_predictions =  bagging_predict2(pred2, X_train_val)
mape_predictions =  bagging_predict2(pred3, X_train_val)
mae_predictions = bagging_predict2(pred4, X_train_val)
mse_predictions =  bagging_predict2(pred5, X_train_val)


concat_G = np.concatenate([smape_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
sMAPE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mape_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MAPE= np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mase_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MASE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mae_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MAE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)

concat_G = np.concatenate([mse_predictions],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
MSE = np.sqrt(mean_squared_error(y_train_val.flatten(),fin_pred_G.flatten())).round(5)


MSE, MASE, MAE, MAPE, sMAPE

5/5 [==============================] - 1s 89ms/step


(2.62788, 2.69738, 2.60468, 16.96598, 14.82846)

In [7]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions2 = bagging_predict2(pred1, test_X)
smape_predictions2 = np.median(np.concatenate([smape_predictions2],axis=0),axis=0)

mase_predictions2 =bagging_predict2(pred2, test_X)
mase_predictions2 = np.median(np.concatenate([mase_predictions2],axis=0),axis=0)

mape_predictions2 =bagging_predict2(pred3, test_X)
mape_predictions2 = np.median(np.concatenate([mape_predictions2],axis=0),axis=0)

mae_predictions2 = bagging_predict2(pred4,test_X)
mae_predictions2 = np.median(np.concatenate([mae_predictions2],axis=0),axis=0)

mse_predictions2 =bagging_predict2(pred5,test_X)
mse_predictions2 = np.median(np.concatenate([mse_predictions2],axis=0),axis=0)


#concat_mase = np.concatenate([np.nan_to_num(np.array(mase_predictions2), nan=0)])
#fin_pred_mase = np.median(concat_mase,axis=1)

#concat_mape = np.concatenate([np.nan_to_num(np.array(mape_predictions2), nan=0)])
#fin_pred_mape = np.median(concat_mape,axis=1)

#concat_smape = np.concatenate([np.nan_to_num(np.array(smape_predictions2), nan=0)])
#fin_pred_smape = np.median(concat_smape,axis=1)

#concat_mae = np.concatenate([np.nan_to_num(np.array(mae_predictions2), nan=0)])
#fin_pred_mae = np.median(concat_mae,axis=1)

#concat_mse = np.concatenate([np.nan_to_num(np.array(mse_predictions2), nan=0)])
#fin_pred_mse = np.median(concat_mse,axis=1)

performance = np.array([MAE, MAPE,sMAPE,MSE,MASE])
beta = 3 # 조정 파라미터
weights = np.exp(-beta * performance)

gd= np.concatenate([mae_predictions2,
                    mape_predictions2,
                   smape_predictions2,
                   mse_predictions2,
                   mase_predictions2],axis=0)
#gd=np.median(gd,axis=2)
normalized_weights = weights / np.sum(weights)

# 각 모델의 예측값에 가중치를 부여하여 앙상블 예측 생성
ensemble_prediction = np.dot(normalized_weights, gd.reshape(5,-1))
print('############################################################################################') 
print('############################################################################################') 
pd.DataFrame(ensemble_prediction.flatten()).to_csv('exp7/Transformer.csv')


print('exp',np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten())).round(5))

12/12 [==============================] - 1s 93ms/step
############################################################################################
############################################################################################
exp 2.7932


In [8]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions_G = bagging_predict2(pred1, test_X)
mase_predictions_G = bagging_predict2(pred2,test_X)
mape_predictions_G = bagging_predict2(pred3,test_X)
mae_predictions_G = bagging_predict2(pred4, test_X)
mse_predictions_G = bagging_predict2(pred5,test_X)


print('############################################################################################') 
print('############################################################################################') 

concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G,mae_predictions_G,mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('all',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5)) 

concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('original',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mse_predictions_G, mase_predictions_G,mae_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('best',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mse',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))


concat_G = np.concatenate([mase_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mase',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mae_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mae',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mape',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([smape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('smape',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

12/12 [==============================] - 1s 94ms/step
############################################################################################
############################################################################################
all 3.02738
original 10.80081
best 2.79853
mse 2.86341
mase 2.82237
mae 2.84423
mape 17.99409
smape 15.60968


In [9]:
eee = []
for i in range(1,20):
    weights = np.exp(-i* performance)
    normalized_weights = weights / np.sum(weights)
    ensemble_prediction = np.dot(normalized_weights, gd.reshape(5,-1))
    eee.append((i,np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten().round(5)))))
    #print(f'exp_beta{i}',np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten())).round(5))

eee

[(1, 2.7908297),
 (2, 2.7919798),
 (3, 2.7931993),
 (4, 2.794474),
 (5, 2.7957914),
 (6, 2.797143),
 (7, 2.7985168),
 (8, 2.7999039),
 (9, 2.8012946),
 (10, 2.8026807),
 (11, 2.8040547),
 (12, 2.8054101),
 (13, 2.8067403),
 (14, 2.8080406),
 (15, 2.8093066),
 (16, 2.8105352),
 (17, 2.8117237),
 (18, 2.8128693),
 (19, 2.8139715)]